# Cleaning and Wrangling: Seattle Open Street Map

We now analyze Open Street Map for the City of Seattle.

## Explore a Subset of Data

Due to the size of the dataset, we need a way to systematically slice the original dataset for a workable sample to explore. To this end, I have used the following code to achieve this. The **k** value is changed from large to small so that my resulting 
*SAMPLE_FILE* ends up at different sizes. When starting out, try using a larger k, then move on to an intermediate k before processing your whole dataset.

In [4]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow
import xml.etree.cElementTree as ET
import pprint
import pickle
from collections import defaultdict
import re

In [9]:
OSM_FILE = "seattle_washington.osm"  # Replace this with your osm file
SAMPLE_FILE = "test.osm"

In [ ]:
k = 5000 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

In [ ]:
output.close()

At the end of the above code, we end up with a file *test.osm* with which we can use to explore the dataset. 

### Develop a Dictionary for All Tags In the Original Dataset

Our goal here is to end up with a Python dictionary for the tags in the original dataset, so that we know what needs to be wrangled in the data. The following achives this.

In [2]:
def count_tags(filename):
    tags = {}
    
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            tags[elem.tag] = 1
            if 
        else:
            tags[elem.tag] += 1
    
    return tags

In [4]:
tags = count_tags('seattle_washington.osm')

In [5]:
import pickle 

with open('tags.pickle', 'wb') as tagsPickle:
    pickle.dump(tags, tagsPickle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('tags.pickle', 'rb') as tagsPickle:
    unserialized_tags = pickle.load(tagsPickle)


In [7]:
unserialized_tags

{'bounds': 1,
 'member': 88068,
 'nd': 8453162,
 'node': 7580046,
 'osm': 1,
 'relation': 9411,
 'tag': 4708553,
 'way': 750242}

### Exploring What Is Contained Within Each Tag Type

To get a better sense of what sort of attributes is contained inside each type of tag, we use the following code to return this information to us.

In [4]:
bounds_subtags = []
member_subtags = []
nd_subtags = []
node_subtags = []
osm_subtags = []
relation_subtags = []
tag_subtags = []
way_subtags = []

for _, element in ET.iterparse('seattle_washington.osm'):
    if element.tag == 'bounds' and element.attrib.keys() not in bounds_subtags:
        bounds_subtags.append(element.attrib.keys())
    elif element.tag == 'member' and element.attrib.keys() not in member_subtags:
        member_subtags.append(element.attrib.keys())
    elif element.tag == 'nd' and element.attrib.keys() not in nd_subtags:
        nd_subtags.append(element.attrib.keys())
    elif element.tag == 'node' and element.attrib.keys() not in node_subtags:
        node_subtags.append(element.attrib.keys())
    elif element.tag == 'osm' and element.attrib.keys() not in osm_subtags:
        osm_subtags.append(element.attrib.keys())
    elif element.tag == 'relation' and element.attrib.keys() not in relation_subtags:
        relation_subtags.append(element.attrib.keys())
    elif element.tag == 'tag' and element.attrib.keys() not in tag_subtags:
        tag_subtags.append(element.attrib.keys())
    elif element.tag == 'way' and element.attrib.keys() not in way_subtags:
        way_subtags.append(element.attrib.keys())
    else:
        pass

In [5]:
bounds_subtags

[dict_keys(['maxlon', 'maxlat', 'minlat', 'minlon'])]

In [6]:
member_subtags

[dict_keys(['type', 'ref', 'role'])]

In [7]:
nd_subtags

[dict_keys(['ref'])]

In [8]:
node_subtags

[dict_keys(['lon', 'version', 'changeset', 'lat', 'timestamp', 'id', 'uid', 'user']),
 dict_keys(['lon', 'version', 'changeset', 'lat', 'timestamp', 'id'])]

In [9]:
osm_subtags

[dict_keys(['generator', 'version', 'timestamp'])]

In [10]:
relation_subtags

[dict_keys(['version', 'changeset', 'timestamp', 'id', 'uid', 'user'])]

In [11]:
way_subtags

[dict_keys(['version', 'changeset', 'timestamp', 'id', 'uid', 'user'])]

In [12]:
tag_subtags

[dict_keys(['v', 'k'])]

### Audit Plan: Addresses

From a visual inspection of the subset of Seattle OSM, we get the sense that **<tag>** contains address information. In particular, tags with attribute of  **k** of **addr:street** contains street names that tend to be described inconsistently in the dataset. Therefore, our next goal is to develop a data audit plan that works specifically on tags with addresses.

The following chuncks of code achive this goal.

In [3]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [4]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [11]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [12]:
def audit(osmfile):
    osm_file = open('seattle_washington.osm', 'r', encoding='cp1252', errors='replace')
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [10]:
st_types = audit(OSM_FILE)

In [13]:
st_types

defaultdict(set,
            {'1': {'228th St SE Suite 1', 'Southeast 132nd Street #1'},
             '100': {'Northwest Byron Street #100',
              'Old Highway 9 Southwest  #100',
              'S 196th St #100',
              'Southeast 38th Street Suite 100'},
             '101': {'156th Street East #101',
              '5th Street #101',
              'East Highway 101',
              'East US Highway 101'},
             '102': {'15th Street Southwest  #102'},
             '104': {'Northeast State Highway 104', 'State Highway 104'},
             '105': {'State Route 105'},
             '110': {'Northeast 4th Street, Suite 110'},
             '1109': {'NE Northgate Way #1109'},
             '112': {'Craftsman Way Suite 112'},
             '11th': {'South 11th'},
             '12': {'HWY 12', 'State Route 12', 'US Highway 12'},
             '125': {'Better Way SE Ste 125'},
             '12th': {'South 12th'},
             '13th': {'South 13th'},
             '140': {'Highland

The above Python dictionary shows the entire collection of street types after we have done our initial cleaning. Now we see that a vast majority of street types no longer bears problems. However, some of the street types are obviously wrong. Most notably, whenever **Suite number / apartment number** is present in the street name, the code has confused it with the name of the street. This needs further cleaning.

In [29]:
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Ave.": "Avenue"
          }

In [15]:
def update_name(name, mapping):
    m = street_type_re.search(name)
    street_type = m.group()
    
    name = re.sub(street_type, mapping[street_type], name)
    return name

{'Ave': 'Avenue', 'Rd.': 'Road', 'St': 'Street', 'St.': 'Street'}

## Preparing the Data for Dababase Insertion

We can describe our audit plan so far as this:

#. 

Now we are ready to insert the cleaned and wrangled data into the MongoDB database.

In [33]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    position_attributes = ['lat', 'lon']
    created_attributes = CREATED

    if element.tag == "node" or element.tag == "way":
        # populate tag type
        node['type'] = element.tag

        # initialize address
        address = {}

        # parse through attributes
        for attribute in element.attrib:
            if attribute in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][attribute] = element.get(attribute)
            elif attribute in position_attributes:
                continue
            else:
                node[attribute] = element.get(attribute)

        # populate position
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        # parse second-level tags for nodes
        for child in element:
            # parse second-level tags for ways and populate `node_refs`
            if child.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in child.attrib:
                    node['node_refs'].append(child.get('ref'))

            # throw out not-tag elements and elements without `k` or `v`
            if child.tag != 'tag'\
            or 'k' not in child.attrib\
            or 'v' not in child.attrib:
                continue
            key = child.get('k')
            val = child.get('v')

            # skip problematic characters
            if problemchars.search(key):
                continue

            # parse address k-v pairs
            elif address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val


            # catch-all
            else:
                node[key] = val
        # compile address
        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']
            # parse through address objects
            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = update_name(val, mapping) if val in mapping else val
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = update_name(val, mapping) if val in mapping else val
                else:
                    node['address'][key] = update_name(val, mapping) if val in mapping else val
            # assign street_full or fallback to compile street dict
            if street_full:
                node['address']['street'] = update_name(street_full, mapping) if street_full in mapping else street_full
            elif len(street_dict) > 0:
                node['address']['street'] = ' '.join([street_dict[key] for key in street_format])
        return node
    else:
        return None

In [34]:
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [35]:
data = process_map('seattle_washington.osm')

In [36]:
data

[{'created': {'changeset': '37473849',
   'timestamp': '2016-02-27T00:03:41Z',
   'uid': '2601744',
   'user': 'sctrojan79',
   'version': '4'},
  'id': '21505268',
  'pos': [47.681693, -122.3213797],
  'type': 'node'},
 {'created': {'changeset': '9689672',
   'timestamp': '2011-10-29T23:34:53Z',
   'uid': '6009',
   'user': 'CoreyBurger',
   'version': '4'},
  'id': '25832758',
  'pos': [48.4364741, -123.3129318],
  'type': 'node'},
 {'created': {'changeset': '214775',
   'timestamp': '2007-02-12T00:44:16Z',
   'uid': '6009',
   'user': 'CoreyBurger',
   'version': '1'},
  'id': '25832786',
  'pos': [48.4350605, -123.3129725],
  'type': 'node'},
 {'created': {'changeset': '214775',
   'timestamp': '2007-02-12T00:44:21Z',
   'uid': '6009',
   'user': 'CoreyBurger',
   'version': '1'},
  'id': '25832792',
  'pos': [48.4351886, -123.3129511],
  'type': 'node'},
 {'created': {'changeset': '9689672',
   'timestamp': '2011-10-29T23:34:53Z',
   'uid': '6009',
   'user': 'CoreyBurger',
   've

## Inserting Into MongoDB Database

After this step, we then insert into the MongoDB database using the following cmd command:

*mongoimport -host 127.0.0.1:27017 --db osm --collection seattle_osm --drop --file C:\Users\Jenny\Documents\Mathfreak_Data\School\Data_Analysis_ND\Project3\seattle_washingotn.osm.json*

## Exploring the Database in MongoDB

### Overview of Seattle Area Map

In [5]:
from pymongo import MongoClient

def get_db():
    client = MongoClient('localhost:27017')
    db = client.osm
    return db

In [7]:
db = get_db()
db.seattle_osm.find().count()      

8330288

In [10]:
db.seattle_osm.find({"type":"node"}).count()

7580018

In [11]:
db.seattle_osm.find({"type":"way"}).count()

750175

In [18]:
len(db.seattle_osm.distinct("created.user"))

3280

### Number of Distinct Points Contained in Map

In [220]:
def make_pipeline():
    pipeline = [ ]
    #group = {'$group':{'_id':'$pos', 'count': { '$sum': 1 }, 'uniq_pos':{'$addToSet':'$_id'}}}
    group = {'$group':{'_id':'$pos', 'uniq_count': { '$sum': 1 }}}
    sort = {'$sort':{'count':-1}}
    #project = {'$project':{'count':'$uniq_pos'}}
    #group1 = {'$group':{'_id':None, 'totalUniq': { '$addToSet': '$uniq_pos' }, 'count': { '$sum': 1 }}}
    group1 = {'$count':'uniq_count'}
    
    for e in [group, sort, group1]:
        pipeline.append(e)
    
    return pipeline

In [221]:
def aggregate(db, pipeline):
    return [doc for doc in db.seattle_osm.aggregate(pipeline)]

In [222]:
pipeline = make_pipeline()
#esult = aggregate(db, pipeline)
pos_result = db.seattle_osm.aggregate(pipeline, allowDiskUse=True)
#result = work1.aggregate(pipe, allowDiskUse=True).


In [223]:
for p in pos_result:
    print(p)

{'uniq_count': 7574033}


### Top 10 Amenities in Metro Seattle 

In [17]:
amenity_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, 
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [18]:
for a in amenity_result:
    print(a)

{'count': 10753, '_id': 'parking'}
{'count': 3318, '_id': 'bicycle_parking'}
{'count': 3293, '_id': 'restaurant'}
{'count': 2848, '_id': 'bench'}
{'count': 2458, '_id': 'school'}
{'count': 1645, '_id': 'place_of_worship'}
{'count': 1578, '_id': 'fast_food'}
{'count': 1499, '_id': 'cafe'}
{'count': 1294, '_id': 'waste_basket'}
{'count': 1153, '_id': 'fuel'}


### Top 10 Place of Worship

In [19]:
religion_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
                                                {"$group":{"_id":"$religion", "count":{"$sum":1}}},
                                                {"$sort":{"count":-1}}, {"$limit":10}])

In [20]:
for r in religion_result:
    print(r)

{'count': 1500, '_id': 'christian'}
{'count': 80, '_id': None}
{'count': 20, '_id': 'jewish'}
{'count': 17, '_id': 'buddhist'}
{'count': 7, '_id': 'muslim'}
{'count': 6, '_id': 'unitarian_universalist'}
{'count': 3, '_id': 'sikh'}
{'count': 2, '_id': 'bahai'}
{'count': 2, '_id': 'eckankar'}
{'count': 2, '_id': 'spiritualist'}


### Top 10 Dining Choice

In [22]:
dinning_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}}, 
                          {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [23]:
for d in dinning_result:
    print(d)

{'count': 817, '_id': None}
{'count': 265, '_id': 'mexican'}
{'count': 257, '_id': 'pizza'}
{'count': 249, '_id': 'american'}
{'count': 162, '_id': 'asian'}
{'count': 155, '_id': 'thai'}
{'count': 150, '_id': 'chinese'}
{'count': 123, '_id': 'japanese'}
{'count': 117, '_id': 'italian'}
{'count': 101, '_id': 'burger'}


### Top 10 Types of Coffee Shops

In [27]:
cafe_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"cafe"}}, 
                          {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [28]:
for c in cafe_result:
    print(c)

{'count': 663, '_id': 'coffee_shop'}
{'count': 567, '_id': None}
{'count': 48, '_id': 'ice_cream'}
{'count': 20, '_id': 'sandwich'}
{'count': 19, '_id': 'american'}
{'count': 15, '_id': 'tea'}
{'count': 14, '_id': 'donut;coffee_shop'}
{'count': 10, '_id': 'vietnamese'}
{'count': 10, '_id': 'donut'}
{'count': 9, '_id': 'frozen_yogurt'}
